In [ ]:
from glob import glob
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
amino_acids_short = {"ALA":"A", "ARG":"R", "ASN":"N", "ASP":"D", "CYS":"C", "GLU":"E", "GLN":"Q", "GLY":"G", "HIS":"H", "ILE":"I", "LEU":"L", "LYS":"K", "MET":"M", "PHE":"F", "PRO":"P", "SER":"S", "THR":"T", "TRP":"W", "TYR":"Y", "VAL":"V", "SEC":"U", "PYL":"O"}

In [ ]:
pdbs = glob("/home/dlsrnsi/DTI/scPDB/scPDB/*/")

In [ ]:
from Bio import PDB

In [ ]:
pdb_parser = PDB.PDBParser(QUIET=True)

In [ ]:
pdb_path = "./scPDB/"

In [ ]:
def get_protein(scpdb):
    try:
        pdb_name = scpdb.split("_")[0]
        structure = pdb_parser.get_structure(pdb_name, pdb_path+scpdb+"/protein.pdb")
        chain_name = list(structure[0].child_dict.keys())[0]
        chain = structure[0][chain_name]
        residues = [residue for residue in chain.get_residues() if residue.resname in amino_acids_short.keys()]
        pdb_sequence = "".join([amino_acids_short[residue.resname] for residue in residues])
        residue_number = [residue.get_id()[1] for residue in residues]
        return chain_name, pdb_sequence, residue_number
    except Exception as e:
        print(scpdb,e)
        return None

In [ ]:
scpdbs = [scpdb.split("/")[-2] for scpdb in pdbs]

In [ ]:
scpdb_df = pd.DataFrame({"scPDB":scpdbs})

In [ ]:
from multiprocessing import Process, Queue, Pool
def get_pdb_info_bulk(df):
    return df.scPDB.map(get_protein)

def parallelize_dataframe(df, func, num_partitions=50):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, df_split)
    pool.close()
    pool.join()
    return results

In [ ]:
pdb_info_results = parallelize_dataframe(scpdb_df, get_pdb_info_bulk)

In [ ]:
pdb_info_results = pd.concat(pdb_info_results)

In [ ]:
scpdb_df["chain"] = pdb_info_results.map(lambda a: a[0] if a is not None else None)

In [ ]:
scpdb_df["PDB_Sequence"] = pdb_info_results.map(lambda a: a[1] if a is not None else None)

In [ ]:
scpdb_df["PDB_Sequence_index"] = pdb_info_results.map(lambda a: a[2] if a is not None else None)

In [ ]:
from urllib.request import urlopen
import urllib
import json
from io import StringIO

In [ ]:
def get_uniprot(row):
    try:
        pdb = row.PDB
        chain = row.chain
        opened = urllib.request.urlopen("http://www.bioinf.org.uk/servers/pdbsws/query.cgi?plain=2&qtype=pdb&id=%s&chain=%s&"%(pdb, chain))
        read = opened.read()
        chain_dic = json.load(StringIO(read.decode("utf-8")))
        acc = chain_dic['pdbsws'][0]["AC"]
        opened.close()
        return acc
    except Exception as e:
        print(row.PDB, e)
        return None

In [ ]:
scpdb_df["PDB"] = scpdb_df.scPDB.map(lambda a: a.split("_")[0])

In [ ]:
get_uniprot(scpdb_df.loc[0])

In [ ]:
scpdb_df = scpdb_df.dropna()

In [ ]:
def get_uniprot_id_bulk(df):
    return df.apply(get_uniprot, axis=1)

In [ ]:
uniprot_ids = parallelize_dataframe(scpdb_df, get_uniprot_id_bulk, num_partitions=10)

In [ ]:
scpdb_df["Protein_ID"] =  pd.concat(uniprot_ids)

In [ ]:
scpdb_df = scpdb_df.loc[scpdb_df.Protein_ID.isna()==False]

In [ ]:
protein_ids = scpdb_df.Protein_ID.unique()

In [ ]:
import urllib
uniprot_url = "https://www.uniprot.org/uniprot/{0}.fasta"
from Bio import Entrez
 
def get_protein_sequence(protein_id):
    try:
        return get_seq_from_uniprot_acc(protein_id)
    except:
        print(protein_id)
        return get_seq_from_genepept(protein_id)
 
 
def get_seq_from_uniprot_acc(uniprot_acc):
    opened = urllib.request.urlopen(uniprot_url.format(uniprot_acc))
    lines = opened.readlines()
    return "".join([line.decode("utf-8").rstrip() for line in lines[1:]])
 
def get_seq_from_genepept(protein_id):
    try:
        fetched = Entrez.efetch(db="protein",id=protein_id, rettype="fasta", retmode="text")
        fasta = fetched.read().splitlines()
        fasta = "".join(list(filter(None, fasta))[1:])
        return fasta
    except:
        print(protein_id+' is not found')
        return None

In [ ]:
def get_protein_bulk_sequence(protein_ids):
    return {protein_id:get_protein_sequence(protein_id) for protein_id in protein_ids}

def parallelize_function(array, func, num_partitions=50):
    array_split = np.array_split(array, num_partitions)
    pool = Pool(num_partitions)
    results = pool.map(func, array_split)
    pool.close()
    pool.join()
    return results

In [ ]:
len(protein_ids)

In [ ]:
sequences = parallelize_function(protein_ids, get_protein_bulk_sequence, 20)

In [ ]:
sequence_dic = {}
for sequence in sequences:
    sequence_dic.update(sequence)

In [ ]:
scpdb_df["Sequence"] = scpdb_df.Protein_ID.map(lambda a: sequence_dic[a])

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

In [ ]:
scpdb_df.head()

In [ ]:
def get_ligand(row):
    try:
        scpdb = row.scPDB
        mol = Chem.SDMolSupplier(pdb_path+scpdb+"/ligand.sdf")[0]
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        pdb = pdb_parser.get_structure(scpdb, pdb_path+scpdb+"/ligand.pdb")
        ligand_name = list(pdb[0].get_residues())[0].resname
        return ligand_name, Chem.MolToSmiles(mol)
    except Exception as e:
        print(scpdb, e)

In [ ]:
ligands = scpdb_df.apply(get_ligand, axis=1)

In [ ]:
scpdb_df["Compound_ID"] = ligands.map(lambda a: a[0] if a is not None else None)

In [ ]:
scpdb_df["SMILES"] = ligands.map(lambda a: a[1] if a is not None else None)

In [ ]:
scpdb_df = scpdb_df.dropna()

In [ ]:
scpdb_df["binding_index"] = scpdb_df.apply(get_binding_index, axis=1)

In [ ]:
from biopandas.mol2 import PandasMol2

In [ ]:
bi_df = PandasMol2().read_mol2(pdb_path+"1uh5_2"+"/site.mol2").df

In [ ]:
bi_df = bi_df.drop_duplicates("subst_name")

In [ ]:
def get_binding_index(row):
    try:
        pdb_name = row.PDB
        scpdb = row.scPDB
        bi_df = PandasMol2().read_mol2(pdb_path+scpdb+"/site.mol2").df
        bi_df = bi_df.drop_duplicates("subst_name")
        bi_df = bi_df.loc[bi_df.subst_name.map(lambda a: a[:3] in amino_acids_short.keys())]
        binding_index = [int(residue[3:]) for residue in bi_df.subst_name.unique()]
        return binding_index
    except Exception as e:
        print(scpdb,e)
        return None

In [ ]:
scpdb_df["PDB_binding_index"] = scpdb_df.apply(get_binding_index, axis=1)

In [ ]:
def get_chain_end(row):
    try:
        scpdb = row.scPDB
        pdb_name = scpdb.split("_")[0]
        structure = pdb_parser.get_structure(pdb_name, pdb_path+scpdb+"/pdb_file.pdb")
        chain_name = row.chain#list(structure[0].child_dict.keys())[0]
        chain = structure[0][chain_name]
        residues = [residue for residue in chain.get_residues() if residue.resname in amino_acids_short.keys()]
        #pdb_sequence = "".join([amino_acids_short[residue.resname] for residue in residues])
        #residue_number = [residue.get_id()[1] for residue in residues]
        return len(residues)#chain_name, pdb_sequence, residue_number
    except Exception as e:
        print(scpdb,e)
        return None

In [ ]:
def get_chain_end_bulk(df):
    return df.apply(get_chain_end, axis=1)

In [ ]:
chain_ends = parallelize_function(scpdb_df, get_chain_end_bulk, 50)

In [ ]:
scpdb_df["chain_end"] = pd.concat(chain_ends)

In [ ]:
def get_seq_number(seq):
    numbers = []
    i = 0
    for aa in seq:
        numbers.append(i)
        if aa!='-':
            i+=1
    return numbers

def get_index_of_number(numbers, number):
    if number==0:
        for i, j in enumerate(numbers):
            #print(j)
            if j!=0:
                return i-1
    else:
        i = 0
        for i, j in enumerate(numbers):
            if j==number:
                return i

In [ ]:
def get_index_adjusted(row):
    try:
        alignment_result = pairwise2.align.localxx(row["Sequence"], row["PDB_Sequence"])
        orig_seq = alignment_result[0][0]
        orig_seq_number = get_seq_number(orig_seq)
        highest_result = alignment_result[0][1]
        pdb_seq_number = get_seq_number(highest_result)
        moving_ind = 0
        k = 0
        n_gap = 0
        results = []
        is_start_gap = True
        for i, j in zip(orig_seq, highest_result):
            if j!="-":
                is_start_gap = False
                results.append(moving_ind)
            if i=="-":
                moving_ind -= 1
            moving_ind += 1
        binding_index_on_pdb_sequence = [row["PDB_Sequence_index"].index(binding_index) for binding_index in row.PDB_binding_index if binding_index in row.PDB_Sequence_index]
        binding_index_adjusted = [results[index] for index in binding_index_on_pdb_sequence if index < len(results)]
        aa_in_pdb = [row.PDB_Sequence[i] for i in binding_index_on_pdb_sequence]
        aa_in_seq = [row.Sequence[i] for i in binding_index_adjusted]
        n_not_matching = 0
        for a_p, a_s in zip(aa_in_pdb, aa_in_seq):
            if a_p!=a_s:
                n_not_matching +=1
        if n_not_matching>0:
            print(row.PDB, row.name, "shows %d mismatch from %d binding_region"%(n_not_matching, len(binding_index_adjusted)))
            if n_not_matching/len(binding_index_adjusted)>0.5:
                print(row.PDB, "dropping currpted PDBBind")
                return None
        return binding_index_adjusted
    except Exception as e:
        print(row.PDB, e)
        return None

In [ ]:
from Bio import pairwise2

In [ ]:
scpdb_with_single_chain = scpdb_df.loc[scpdb_df.PDB_Sequence.map(len)<=scpdb_df.chain_end]

In [ ]:
scpdb_with_single_chain.shape

In [ ]:
scpdb_with_single_chain["binding_index"] = scpdb_with_single_chain.apply(get_index_adjusted, axis=1)

In [ ]:
def get_index_adjusted_for_multichain(row):
    try:
        pdb_sequence = row.PDB_Sequence[:row.chain_end]
        alignment_result = pairwise2.align.localxx(row["Sequence"], pdb_sequence)
        orig_seq = alignment_result[0][0]
        orig_seq_number = get_seq_number(orig_seq)
        highest_result = alignment_result[0][1]
        pdb_seq_number = get_seq_number(highest_result)
        moving_ind = 0
        k = 0
        n_gap = 0
        results = []
        is_start_gap = True
        for i, j in zip(orig_seq, highest_result):
            if j!="-":
                is_start_gap = False
                results.append(moving_ind)
            if i=="-":
                moving_ind -= 1
            moving_ind += 1
        pdb_binding_index = [binding_index for binding_index in row.PDB_binding_index if binding_index <= row.chain_end]
        binding_index_on_pdb_sequence = [row["PDB_Sequence_index"].index(binding_index) for binding_index in pdb_binding_index if binding_index in row.PDB_Sequence_index]
        binding_index_adjusted = [results[index] for index in binding_index_on_pdb_sequence if index < len(results)]
        aa_in_pdb = [row.PDB_Sequence[i] for i in binding_index_on_pdb_sequence]
        aa_in_seq = [row.Sequence[i] for i in binding_index_adjusted]
        n_not_matching = 0
        for a_p, a_s in zip(aa_in_pdb, aa_in_seq):
            if a_p!=a_s:
                n_not_matching +=1
        if n_not_matching>0:
            print(row.PDB, row.name, "shows %d mismatch from %d binding_region"%(n_not_matching, len(binding_index_adjusted)))
            if n_not_matching/len(binding_index_adjusted)>0.5:
                print(row.PDB, "dropping currpted PDBBind")
                return None
        return binding_index_adjusted
    except Exception as e:
        print(row.PDB, e)
        return None

In [ ]:
scpdb_with_multi_chain = scpdb_df.loc[scpdb_df.PDB_Sequence.map(len)>scpdb_df.chain_end]

In [ ]:
scpdb_with_multi_chain["binding_index"] = scpdb_df.apply(get_index_adjusted_for_multichain, axis=1)

In [ ]:
scpdb_full = pd.concat([scpdb_with_single_chain, scpdb_with_multi_chain])

In [ ]:
scpdb_full = scpdb_full.dropna()

In [ ]:
scpdb_full = scpdb_full.loc[scpdb_full.binding_index.map(len)>0]

In [ ]:
def get_binding_region_from_index(row):
    binding_index = np.sort(row.binding_index)
    if len(binding_index)==0:
        return []
    protein_len = len(row.Sequence)
    first_binding_start = binding_index[0] - 2
    first_binding_end = binding_index[0] + 3
    binding_region = [[first_binding_start, first_binding_end]]
    for i in range(1,len(binding_index)):
        binding_region_comp = binding_region[-1]
        if binding_index[i] is None:
            continue
        binding_start  = binding_index[i] - 2
        binding_end  = binding_index[i] + 3
        if binding_region_comp[1] >= binding_start:
            binding_region[-1] = [binding_region_comp[0], binding_end]
        else:
            binding_region.append([binding_start, binding_end])
    binding_region_modified = []
    if binding_region[0][0]<0:
        binding_region[0][0] = 0
    if binding_region[-1][-1]>protein_len:
        binding_region[-1][-1] = protein_len
    return binding_region

In [ ]:
scpdb_full["binding_region"] = scpdb_full.apply(get_binding_region_from_index, axis=1)

In [ ]:
scpdb_full

In [216]:
scpdb_full.to_csv("./scPDB_HoTS.tsv", sep='\t', index=False)